In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from IPython.display import Markdown, clear_output


def get_docs_path():
    """Returns the docs directory used to generate hugo webiste content.  The directory 
    path is determined by relative reference to the file containing this code"""
    return(os.path.join(os.path.dirname(os.getcwd()), 'content', 'docs'))

           
def delete_directory_if_it_exists(dir_to_delete):
    """Deletes directory dir_to_delete and its contents if it exists"""
    if os.path.isdir(dir_to_delete):
        shutil.rmtree(dir_to_delete)
    

def create_formula_directories(docs_dir, sub_paths):
    """Creates 2 directories under docs directory namely 'formulas by year'
    and 'formulas cumulative'.  Under each of these directories create subdirectories for each item in
    sub_paths where sub_paths is a numpy array containing different levels of directory path"""

    formulas_by_year_path = docs_dir + os.path.sep + 'formulas by year'
    formulas_cumulative_path = docs_dir + os.path.sep + 'formulas cumulative'
    for path in sub_paths:
        os.makedirs(formulas_by_year_path + os.path.sep + os.path.sep.join(path))
        os.makedirs(formulas_cumulative_path + os.path.sep + os.path.sep.join(path))

    
def create_formula_index_files(docs_dir, df_sort_orders):
    """Creates _index.md files in relevant folder under root_dir to manage nested website menu """
    
    formulas_by_year_path = docs_dir + os.path.sep + 'formulas by year'
    formulas_cumulative_path = docs_dir + os.path.sep + 'formulas cumulative'
    
    with open(formulas_by_year_path + os.path.sep + '_index.md', "w") as text_file:
        text_file.write("---\n" + 
                        "bookCollapseSection: true\n" +
                        "weight: 1\n" +
                        "---\n\n" +
                       "This section of the website includes current year / course formulas only.")

    with open(formulas_cumulative_path + os.path.sep + '_index.md', "w") as text_file:
        text_file.write("---\n" + 
                        "bookCollapseSection: true\n" +
                        "weight: 2\n" +
                        "---\n\n" +
                       "This section of the website includes both formulas covered in current and previous years \ courses")

    for root,dirs,files in os.walk(formulas_by_year_path):
        #Exclude formulas_by_year_path as index file is already created above
        if root != formulas_by_year_path:
            sort_order = df_sort_orders[df_sort_orders['sort value'] == os.path.basename(root)].index.tolist()[0]
            with open(root + os.path.sep + '_index.md', "w") as text_file:
                text_file.write("---\n" + 
                                "bookCollapseSection: true\n" + 
                                "weight: " + str(sort_order) + "\n" +
                                "---")

    for root,dirs,files in os.walk(formulas_cumulative_path):
        #Exclude formulas_cumulative_path as index file is already created above
        if root!=formulas_cumulative_path:
            sort_order = df_sort_orders[df_sort_orders['sort value'] == os.path.basename(root)].index.tolist()[0]
            with open(root + os.path.sep + '_index.md', "w") as text_file:
                text_file.write("---\n" + 
                                "bookCollapseSection: true\n" + 
                                "weight: " + str(sort_order) + "\n" +
                                "---")

def create_formula_files(docs_dir, df_formulas):
    """Creates formula files in markdown format.  Files are created per state / subject  / category
    combination and stored in folders under docs_path according to this same combination"""

    formula_combination = df_formulas[['State', 'Subject code', 'Category']].drop_duplicates()

    for index, row in formula_combination.iterrows():
        formula_set_df = formulas_by_state_subject_category_df(df_formulas, row['State'], row['Subject code'], row['Category'])
        formula_set_styler = df_to_formula_styled_table(df=formula_set_df,col_widths={'Formula_1':300, 'Formula_2':400},
                                                        display_col_headers = False)
        output_string =  '#  \n<br>\n' + formula_set_styler.to_html()
        file_name = docs_dir + os.path.sep +'formulas by year' \
            + os.path.sep  + row['State'] \
            + os.path.sep  + str(row['Subject code']) \
            + os.path.sep  + row['Category']  + '.md'
        with open(file_name, "w") as text_file:
            text_file.write(output_string)


def formulas_by_state_subject_category_df(df_formulas, state, subject_code, category):
    """returns all formulas for given state, subject and category, returns formula_1 column and formula_2 column if not empty
    The return value is a pandas dataframe"""

    df_filtered =  df_formulas[
        (df_formulas['State'] == state) &
        (df_formulas['Subject code'] == str(subject_code)) & 
        (df_formulas['Category'] == category)]   
    formula_2_col_is_empty = df_filtered['Formula_2'].dropna().empty
    
    if formula_2_col_is_empty:
        df_filtered = df_filtered[['Formula_1']]
    else:
        df_filtered = df_filtered[['Formula_1', 'Formula_2']]

    return (df_filtered)
        

def df_calculus_summary(df_formulas):
    """Returns a summary of derivative and integral formulasas a pandas dataframe"""
    
    df_calculus = df_formulas[['Category', 'Group', 'Formula_1', 'Formula_2', 'Comment']][df_formulas["Category"].isin(["Differentiation","Integration"])]
    df_calculus = df_calculus.pivot(columns='Category', index = 'Group').fillna('')
    
    # Flatten the multi-index headings after pivot
    df_calculus.columns = df_calculus.columns.get_level_values(0) +' ' + df_calculus.columns.get_level_values(1)
    df_calculus = df_calculus.reset_index()

    df_calculus['Comment'] = df_calculus.apply(_calclus_summary_comment, axis=1)
    
    df_calculus = df_calculus.sort_values(by='Group')
    df_calculus =  df_calculus.drop(labels = ['Group', 'Comment Differentiation', 'Comment Integration', 'Formula_2 Integration'], axis = 1)
    df_calculus = df_calculus.rename(columns={
        "Formula_1 Differentiation": "Function", 
        "Formula_1 Integration":"Equivalent integral",
        "Formula_2 Differentiation": "Derivative"})

    # Reorder columns
    df_calculus = df_calculus[['Function', 'Derivative', 'Equivalent integral', 'Comment']]

    return(df_calculus)


def styler_calculus_summary(df_calculus, formula_sheet=[]):
    """Returns a pandas styler version of the df_calculus dataframe as returned by df_calculus_summary function"""

    if len(formula_sheet):
        styler_calculus = df_to_formula_styled_table(df=df_calculus, 
                                                     col_widths={'Function': 200, 'Derivative': 300,'Equivalent integral': 400,'Comment':600},
                                                     cols_to_highlight_if_in_formula_sheet= {'Derivative', 'Equivalent integral'},
                                                     formula_sheet=formula_sheet)
    else:
        styler_calculus = df_to_formula_styled_table(df=df_calculus, 
                                                     col_widths={'Function': 200, 'Derivative': 300,'Equivalent integral': 400,'Comment':600})
    
    return(styler_calculus)


def _calclus_summary_comment(row):
    """Returns a comment for calculus formula summary based on derivative and integral comments"""
    if row['Comment Differentiation'] == row['Comment Integration']:
        return_value = row['Comment Differentiation']
    elif row['Comment Differentiation'] == '':
        return_value = row['Comment Integration']
    elif row['Comment Integration'] == '':
        return_value = row['Comment Differentiation']            
    else:
        return_value = row['Comment Differentiation'] + '\n' + row['Comment Integration']
    return(return_value)


def set_styled_table_widths(styled_table, widths):
    """Sets pandas dataframe stlyle column withs where widths is represents a dict of column names and widths in pixels as integers
    Ignores column names that do not exist.  Purpously does not generate an error to allow the function to be called for a 
    broader set of column names without requiring a check in the calling sub"""
    return_table = styled_table
    for column_name, width in widths.items():
        if column_name in styled_table.columns:
            return_table = return_table.set_properties(subset=[column_name], **{'width': str(width) + 'px'})
    return(return_table)


def df_to_formula_styled_table(df, col_widths={}, cols_to_highlight_if_in_formula_sheet=[], formula_sheet=[], display_col_headers = True):
    """Converts pandas dataframe to a styler and applies various formatting"""

    styled_table = df.fillna('').style
    styled_table = set_styled_table_widths(styled_table, col_widths)

    if not display_col_headers:
        styled_table = styled_table.hide().hide(axis='columns')

    for col in cols_to_highlight_if_in_formula_sheet:
        styled_table = styled_table.applymap(is_on_formula_sheet_formatting, subset=[col], formula_sheet=formula_sheet) 
    
    styled_table = styled_table.set_table_styles ([
        {'selector': 'th.col_heading', 'props': 'text-align: left; font-size:1em;'},
        {'selector': 'td', 'props': 'text-align: left; font-size:1em;padding: 1.5em;'}]) 

    # below allows newlines in the csv, outside of the latex dollar signs to be reflected on display
    styled_table= styled_table.set_properties(**{'white-space': 'pre-wrap'})
    
    return (styled_table)


def formulas_on_formula_sheet(df_formulas):
    """Returns a list of formulas on formula sheet that returns all fields Formula_1 and Formula_2 of 
    df_Formulas where field 'On formula sheet' is True"""
    formulas_one_on_sheet = df_formulas[df_formulas['On formula sheet'] == True]['Formula_1'].values.tolist()
    formulas_two_on_sheet = df_formulas[df_formulas['On formula sheet'] == True]['Formula_2'].values.tolist()
    formulas_on_sheet = formulas_one_on_sheet + formulas_two_on_sheet
    return (formulas_on_sheet)


def is_on_formula_sheet_formatting(formula, formula_sheet):
    """Returns formatting for pandas styler object based on whether formulas is contained in formula_sheet list"""
    if formula in formula_sheet:
        return ('background-color:rgba(255,194,10, 0.2);')
    else:
        return (None)

        
def get_sort_orders():
    """returns a combined dataframe of sort orders for states, subject codes and categories as per csv files"""
    df_state_order =  pd.read_csv(filepath_or_buffer='order_states.csv', header = 0, names = ['sort value'])
    df_subject_order =  pd.read_csv(filepath_or_buffer='order_subjects.csv', header = 0, names = ['sort value'])
    df_sorted = pd.concat([df_state_order, df_subject_order])
    #Start index to enable use for sorting menus in hugo-book via weight setting (doesnt seem to work with weight starting from zero)
    df_sorted.index+=1
    return(df_sorted)


if __name__ == '__main__':
    
    # # Latex string in csv needs to be enclosed a single $ to enable left align
    # df_formulas =  pd.read_csv(filepath_or_buffer='formulas.csv')
    # df_sort_orders = get_sort_orders()
    
    # docs_dir = get_docs_path()
    # delete_directory_if_it_exists(docs_dir)


    
    # formula_paths = df_formulas[['State', 'Subject code']].drop_duplicates().to_numpy(dtype = str)
    # create_formula_directories(docs_dir, formula_paths)
    # create_formula_index_files(docs_dir, df_sort_orders)
    # create_formula_files(docs_dir, df_formulas)

    # # df_calculus = df_calculus_summary(df_formulas)
    # # styler_calculus = styler_calculus_summary(df_calculus, formulas_on_formula_sheet(df_formulas))
    # # display(styler_calculus)
    

    # # display(get_sort_orders())


FileNotFoundError: [Errno 2] No such file or directory: 'order_states.csv'

In [6]:
import formulas
import folders_and_storage
import pandas as pd

if __name__ == '__main__':
    
    # Latex string in csv needs to be enclosed a single $ to enable left align
    df_formulas =  pd.read_csv(filepath_or_buffer='formulas.csv')
    df_sort_orders = formulas.get_sort_orders()
    
    docs_dir = formulas.get_docs_path()
    formulas.delete_directory_if_it_exists(docs_dir)


    
    # formula_paths = df_formulas[['State', 'Subject code']].drop_duplicates().to_numpy(dtype = str)
    # create_formula_directories(docs_dir, formula_paths)
    # create_formula_index_files(docs_dir, df_sort_orders)
    # create_formula_files(docs_dir, df_formulas)

    df_calculus = formulas.df_calculus_summary(df_formulas)
    styler_calculus = formulas.styler_calculus_summary(df_calculus, formulas.formulas_on_formula_sheet(df_formulas))
    display(styler_calculus)

    # display(get_sort_orders())

,Function,Derivative,Equivalent integral,Comment
0,$ y=f(x) $,$ \dfrac{dy}{dx} =\lim\limits_{h \to 0} \dfrac{f(x+h) - f(x)}{h} $,,
1,$ y=kx $,$ \dfrac{dy}{dx} =k $,,
2,$ y=k $,$ \dfrac{dy}{dx} =0 $,,
3,$ y=x^n $,$ \dfrac{dy}{dx} =nx^{n-1} $,,
4,$ y=kx^n $,$ \dfrac{dy}{dx} =knx^{n-1} $,,
5,$ y=kf(x) $,$ \dfrac{dy}{dx} =kf'(x) $,,
6,$ y=f(x) + g(x) $,$ \dfrac{dy}{dx} =f'(x) + g'(x) $,,
7,$y=f(x)^n$,$\dfrac{dy}{dx}=nf'(x)[f(x)]^{n-1}$,$ {\Large\int} f'(x)[f(x)]^n dx = \dfrac{1}{n+1}[f(x)]^{n+1} + c $ $ \text{ where } n \neq -1 $,When n = -1 follow the integration rules for $ {\Large\int} \dfrac{f'(x)}{f(x)}dx$
8,$y=uv$,$ \dfrac{dy}{dx} = u\dfrac{dv}{dx} + v\dfrac{du}{dx}$,$ {\Large\int} u \dfrac{dv}{dx} dx=uv-{\Large\int}v \dfrac {du}{dx}dx$,
9,$y=\dfrac{u}{v}$,$ \dfrac{dy}{dx} = \dfrac{v\dfrac{du}{dx} - u\dfrac{dv}{dx}}{v^2}$,,
